In [1]:
from pathlib import Path
import pandas as pd
import numpy as np

from sklearn.experimental import enable_halving_search_cv
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import TargetEncoder, OneHotEncoder, StandardScaler, LabelEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, HalvingGridSearchCV
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn import set_config
set_config(enable_metadata_routing=True)

from xgboost import XGBClassifier
from prince import FAMD
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

from time import monotonic

from utils.data_processing import transform_ipinfo

In [ ]:
RANDOM_STATE = 124
data_path = Path("./data")
input_data_path = Path("./data/cybersecurity_attacks.csv")
df = pd.read_csv(input_data_path)

X_orig = df.copy().drop(columns=["Attack Type"])
y_orig = df["Attack Type"].astype("category").cat.codes

ip_features = transform_ipinfo(df[["Source IP Address", "Destination IP Address","Proxy Information"]])
X_orig = X_orig.merge(ip_features, left_index=True, right_index=True, how="left")
X_orig = X_orig.drop(columns=["Source IP Address", "Destination IP Address", "Proxy Information", "Int Proxy Information"
                              "Global Source IP Address", "Global Destination IP Address", "Global Proxy Information"])

In [3]:
X_orig.columns

Index(['Timestamp', 'Source Port', 'Destination Port', 'Protocol',
       'Packet Length', 'Packet Type', 'Traffic Type', 'Payload Data',
       'Malware Indicators', 'Anomaly Scores', 'Alerts/Warnings',
       'Attack Signature', 'Action Taken', 'Severity Level',
       'User Information', 'Device Information', 'Network Segment',
       'Geo-location Data', 'Firewall Logs', 'IDS/IPS Alerts', 'Log Source',
       'Int Source IP Address', 'Int Destination IP Address',
       'Int Proxy Information'],
      dtype='object')

# Component Reduction Before Smote

In [4]:
cat_cols = X_orig.select_dtypes(include="str").columns
num_cols = X_orig.select_dtypes(include="number").columns
passthrough_columns = [col for col in X_orig.columns if col not in cat_cols and col not in num_cols]

# Check cardinality of categorical features
MAX_CARDINALITY = 50  # Threshold for high cardinality

cardinality_info = {}
for col in cat_cols:
    cardinality = X_orig[col].nunique()
    cardinality_info[col] = cardinality

cardinality_df = pd.DataFrame.from_dict(cardinality_info, orient='index', columns=['Cardinality'])
cardinality_df = cardinality_df.sort_values('Cardinality', ascending=False)

print(f"Categorical features with cardinality:")
print(cardinality_df)

# Filter out high cardinality features
high_card_cols = cardinality_df[cardinality_df['Cardinality'] > MAX_CARDINALITY].index.tolist()
print(f"\nRemoving {len(high_card_cols)} high-cardinality features (>{MAX_CARDINALITY} unique values): {high_card_cols}")

# Update cat_cols to exclude high cardinality features
cat_cols = [col for col in cat_cols if col not in high_card_cols]
print(f"\nRemaining categorical features: {len(cat_cols)}")

# Drop high cardinality columns from train and test sets
X_lowcard = X_orig.drop(columns=high_card_cols)

TypeError: numpy string dtypes are not allowed, use 'str' or 'object' instead

In [ ]:
cat_cols = X_lowcard.select_dtypes(include=["object","str"]).columns
num_cols = X_lowcard.select_dtypes(include="number").columns
passthrough_columns = [col for col in X_lowcard.columns if col not in cat_cols and col not in num_cols]


In [ ]:
numeric_transformer = Pipeline(
        steps = [
            ("imputer", SimpleImputer(strategy="mean")),
            ("scaler", StandardScaler())
        ])

cat_transformer = Pipeline([
        ("imputer", SimpleImputer(missing_values=np.nan, strategy="constant", fill_value="unknown")),
        ])

preprocessor = ColumnTransformer(
        transformers=[
            ("cat", cat_transformer, cat_cols),
            ("num", numeric_transformer, num_cols)
        ]).set_output(transform="pandas")


In [ ]:
x_processed = preprocessor.fit_transform(X_lowcard)
cat_cols = x_processed.select_dtypes(include=["object","str"]).columns
num_cols = x_processed.select_dtypes(include="number").columns
passthrough_columns = [col for col in X_lowcard.columns if col not in cat_cols and col not in num_cols]
x_processed[cat_cols] = x_processed[cat_cols].astype("category")
x_processed[num_cols] = x_processed[num_cols].astype("float32")

X_train, X_test, y_train, y_test = train_test_split(x_processed, y_orig, test_size=0.2, stratify=y_orig, random_state=RANDOM_STATE)

In [ ]:
famd = FAMD(n_components=20, n_iter=3, random_state=RANDOM_STATE)
xtrain_famd = famd.fit_transform(x_processed)
famd.eigenvalues_summary

,eigenvalue,% of variance,% of variance (cumulative)
component,,,
0,1.019,14.55%,14.55%
1,1.008,14.40%,28.95%
2,1.005,14.35%,43.30%
3,1.002,14.31%,57.61%
4,0.994,14.20%,71.81%
5,0.994,14.20%,86.00%
6,0.980,14.00%,100.00%
7,0.000,0.00%,100.00%
8,0.000,0.00%,100.00%


In [ ]:
famd.column_contributions_.style.format('{:.0%}')

component,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
variable,,,,,,,,,,,,,,,,,,,,
num__Source Port,13%,11%,26%,1%,34%,2%,12%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%
num__Destination Port,22%,12%,2%,17%,0%,28%,18%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%
num__Packet Length,8%,23%,35%,0%,13%,2%,19%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%
num__Anomaly Scores,15%,16%,6%,16%,0%,37%,11%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%
num__Int Source IP,7%,11%,0%,37%,29%,14%,1%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%
num__Int Destination IP,34%,0%,6%,12%,13%,8%,27%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%
num__Int Proxy Information,1%,26%,25%,17%,10%,10%,12%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%,0%
cat__Action Taken,0%,0%,0%,0%,0%,0%,0%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
cat__Alerts/Warnings,0%,0%,0%,0%,0%,0%,0%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%


In [ ]:
famd.column_coordinates_.style.format('{:.3f}')

component,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
variable,,,,,,,,,,,,,,,,,,,,
num__Source Port,0.137,0.116,0.265,0.012,0.335,0.016,0.120,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
num__Destination Port,0.227,0.119,0.022,0.171,0.000,0.283,0.178,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
num__Packet Length,0.077,0.232,0.353,0.000,0.133,0.015,0.190,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
num__Anomaly Scores,0.149,0.161,0.056,0.160,0.001,0.370,0.103,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
num__Int Source IP,0.073,0.116,0.001,0.374,0.293,0.136,0.007,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
num__Int Destination IP,0.345,0.001,0.061,0.118,0.130,0.077,0.268,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
num__Int Proxy Information,0.011,0.262,0.247,0.166,0.102,0.097,0.115,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
cat__Action Taken,0.000,0.000,0.000,0.000,0.000,0.000,0.000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
cat__Alerts/Warnings,0.000,0.000,0.000,0.000,0.000,0.000,0.000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


We can keep [Source Port, Destination Port, Packet Length, Anomaly Scores, Int Source IP, Int Destination IP, Int Proxy Information]

In [ ]:
contributions = famd.column_contributions_
contributions["total"] = contributions.sum(axis=1)
contributions = contributions.sort_values("total", ascending=False)
contributions = contributions[contributions["total"] > 0.005]
important_features = [feature.replace("cat__","").replace("num__","") for feature in contributions.index.tolist()]
print(f"Number of features contributing more than 0.5%: {len(important_features)}")
print(f"Important features: {important_features}")
nb_components = famd.column_coordinates_.T
nb_components["total"] = nb_components.sum(axis=1)
nb_components = nb_components[nb_components["total"] > 0.01]
nb_components = nb_components.shape[0]


In [ ]:
X_simple = X_orig[important_features]
cat_cols = X_simple.select_dtypes(include=["object","str"]).columns
num_cols = X_simple.select_dtypes(include="number").columns
passthrough_columns = [col for col in X_simple.columns if col not in cat_cols and col not in num_cols]


class TimeTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, name=""):
        self.name = name

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        print(f"{monotonic()- start_time} Starting transformation")
        return X

if not passthrough_columns:
    X_train, X_test, y_train, y_test = train_test_split(X_simple, y_orig, test_size=0.2, stratify=y_orig, random_state=RANDOM_STATE)

    numeric_transformer = Pipeline(
        steps = [
            ("imputer", SimpleImputer(strategy="mean")),
            ("scaler", StandardScaler())
        ])
    cat_transformer = Pipeline([
            ("imputer", SimpleImputer(missing_values=np.nan, strategy="constant", fill_value="unknown")),
            ("encoder", OneHotEncoder(drop="first", handle_unknown="ignore"))
            ])
    preprocessor = ColumnTransformer(
            transformers=[
                ("cat", cat_transformer, cat_cols),
                ("num", numeric_transformer, num_cols)
            ]).set_output(transform="pandas")
    
    smote = SMOTETomek(random_state=RANDOM_STATE, n_jobs=6)
    model = GradientBoostingClassifier(random_state=RANDOM_STATE)

    pipeline = ImbPipeline(
            steps=[
                # ("time", TimeTransformer(name="preprocessing")),
                ("preprocessor", preprocessor),
                ("smote", smote),
                ("classifier", model)
            ])
    param_grid = {
        "classifier__max_depth": [None, 3, 10, 20],
        "classifier__min_samples_split": [2, 10, 30, 100]
    }


    search = HalvingGridSearchCV(pipeline, param_grid, cv=5, factor=3, n_jobs=6, resource="classifier__n_estimators",
                                min_resources=10, max_resources=500, random_state=RANDOM_STATE, verbose=1)
    start_time = monotonic()
    search.fit(X_train, y_train)
    print("Time taken to fit the model: %.2f seconds" % (monotonic() - start_time))
    print("Model score: %.3f" % search.score(X_test, y_test))
    labels = ["DDoS", "Intrusion", "Malware"]
    y_pred = search.predict(X_test)
    print(classification_report(y_pred, y_test, target_names=labels))
    print("Best parameters: ", search.best_params_)


n_iterations: 3
n_required_iterations: 3
n_possible_iterations: 4
min_resources_: 10
max_resources_: 500
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 16
n_resources: 10
Fitting 5 folds for each of 16 candidates, totalling 80 fits
----------
iter: 1
n_candidates: 6
n_resources: 30
Fitting 5 folds for each of 6 candidates, totalling 30 fits
----------
iter: 2
n_candidates: 2
n_resources: 90
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Time taken to fit the model: 228.94 seconds
Model score: 0.333
              precision    recall  f1-score   support

        DDoS       0.21      0.33      0.25      1688
   Intrusion       0.40      0.33      0.36      3183
     Malware       0.39      0.34      0.36      3129

    accuracy                           0.33      8000
   macro avg       0.33      0.33      0.33      8000
weighted avg       0.36      0.33      0.34      8000

Best parameters:  {'classifier__max_depth': 3, 'classifier__min_samples_spli

Does not work this way around.
Will try to over sample first then reduce the number of features

## CORRECTED APPROACH - Key Fixes:
1. **Split data FIRST** (avoid data leakage)
2. **Add proper encoding** (OneHotEncoder for categoricals)
3. **Apply FAMD only on training data**
4. **Select 15-20 features** (not just 7)

In [ ]:
# Step 1: Identify and remove high cardinality features
cat_cols = X_orig.select_dtypes(include=["object","str"]).columns
num_cols = X_orig.select_dtypes(include="number").columns

# Check cardinality of categorical features
MAX_CARDINALITY = 50  # Threshold for high cardinality

cardinality_info = {col: X_orig[col].nunique() for col in cat_cols}
cardinality_df = pd.DataFrame.from_dict(cardinality_info, orient='index', columns=['Cardinality'])
cardinality_df = cardinality_df.sort_values('Cardinality', ascending=False)

print("Categorical features with cardinality:")
print(cardinality_df)

# Filter out high cardinality features
high_card_cols = cardinality_df[cardinality_df['Cardinality'] > MAX_CARDINALITY].index.tolist()
print(f"\nRemoving {len(high_card_cols)} high-cardinality features: {high_card_cols}")

# Drop high cardinality columns
X_clean = X_orig.drop(columns=high_card_cols)

# Update column lists
cat_cols_clean = X_clean.select_dtypes(include=["object","str"]).columns.tolist()
num_cols_clean = X_clean.select_dtypes(include="number").columns.tolist()

print(f"\nCleaned data shape: {X_clean.shape}")
print(f"Categorical: {len(cat_cols_clean)}, Numerical: {len(num_cols_clean)}")

Categorical features with cardinality:
                    Cardinality
Payload Data              40000
Timestamp                 39997
User Information          32389
Device Information        32104
Geo-location Data          8723
Severity Level                3
Protocol                      3
Traffic Type                  3
Action Taken                  3
Network Segment               3
Log Source                    2
Attack Signature              2
Packet Type                   2
Malware Indicators            1
Alerts/Warnings               1
Firewall Logs                 1
IDS/IPS Alerts                1

Removing 5 high-cardinality features: ['Payload Data', 'Timestamp', 'User Information', 'Device Information', 'Geo-location Data']

Cleaned data shape: (40000, 19)
Categorical: 12, Numerical: 7


In [ ]:
# Step 2: SPLIT DATA FIRST - Critical to avoid data leakage!
X_train, X_test, y_train, y_test = train_test_split(
    X_clean, y_orig, test_size=0.2, stratify=y_orig, random_state=RANDOM_STATE
)

In [ ]:
# Step 3: Preprocess for FAMD (impute only - FAMD handles categoricals directly)
numeric_transformer_famd = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="mean")),
        ("scaler", StandardScaler())
    ]
)

cat_transformer_famd = Pipeline([
    ("imputer", SimpleImputer(missing_values=np.nan, strategy="constant", fill_value="unknown")),
])

preprocessor_famd = ColumnTransformer(
    transformers=[
        ("cat", cat_transformer_famd, cat_cols_clean),
        ("num", numeric_transformer_famd, num_cols_clean)
    ]
).set_output(transform="pandas")

# Fit on TRAINING data only
X_train_preprocessed = preprocessor_famd.fit_transform(X_train)

# Convert to proper dtypes for FAMD
cat_cols_proc = X_train_preprocessed.select_dtypes(include=["object", "str"]).columns
num_cols_proc = X_train_preprocessed.select_dtypes(include="number").columns

X_train_preprocessed[cat_cols_proc] = X_train_preprocessed[cat_cols_proc].astype("category")
X_train_preprocessed[num_cols_proc] = X_train_preprocessed[num_cols_proc].astype("float32")

print(f"Preprocessed training data shape: {X_train_preprocessed.shape}")

Preprocessed training data shape: (32000, 19)


In [ ]:
# Step 4: Apply FAMD on TRAINING data only
famd_corrected = FAMD(n_components=15, n_iter=3, random_state=RANDOM_STATE)
famd_corrected.fit(X_train_preprocessed)

# Check explained variance
print("Explained variance by component:")
print(famd_corrected.eigenvalues_summary)
print(f"\nCumulative variance explained: {famd_corrected.eigenvalues_summary['% of variance (cumulative)'].iloc[-1]}")


Explained variance by component:
          eigenvalue % of variance % of variance (cumulative)
component                                                    
0              1.018        14.55%                     14.55%
1              1.010        14.42%                     28.97%
2              1.006        14.37%                     43.34%
3              1.001        14.30%                     57.64%
4              0.995        14.21%                     71.85%
5              0.989        14.13%                     85.99%
6              0.981        14.01%                    100.00%
7              0.000         0.00%                    100.00%
8              0.000         0.00%                    100.00%
9              0.000         0.00%                    100.00%
10             0.000         0.00%                    100.00%
11             0.000         0.00%                    100.00%
12             0.000         0.00%                    100.00%
13             0.000         0.00%   

In [ ]:
# Step 5: Visualize column contributions
famd_corrected.column_contributions_.style.format('{:.2%}')

component,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
variable,,,,,,,,,,,,,,,
num__Source Port,6.05%,8.43%,24.68%,0.24%,60.51%,0.01%,0.08%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
num__Destination Port,23.84%,17.28%,6.29%,11.09%,2.45%,11.75%,27.31%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
num__Packet Length,5.16%,32.37%,22.12%,0.02%,0.12%,27.49%,12.72%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
num__Anomaly Scores,7.12%,28.18%,9.66%,11.53%,0.28%,41.49%,1.74%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
num__Int Source IP,11.45%,12.11%,15.48%,17.36%,20.61%,13.25%,9.75%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
num__Int Destination IP,41.29%,0.59%,0.00%,11.41%,7.37%,0.69%,38.65%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
num__Int Proxy Information,5.09%,1.04%,21.77%,48.36%,8.66%,5.33%,9.75%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
cat__Action Taken,0.00%,0.01%,0.01%,0.01%,0.01%,0.01%,0.01%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%
cat__Alerts/Warnings,0.01%,0.01%,0.00%,0.01%,0.00%,0.00%,0.01%,nan%,nan%,nan%,nan%,nan%,nan%,nan%,nan%


In [ ]:
# Step 6: Select important features based on total contribution
# Use a lower threshold to get more features (15-20 instead of just 7)
contributions = famd_corrected.column_contributions_.copy()
contributions["total"] = contributions.sum(axis=1)
contributions = contributions.sort_values("total", ascending=False)

# Lower threshold from 0.01 to 0.005 to get more features
contributions_filtered = contributions[contributions["total"] > 0.005]

# Extract original feature names (remove preprocessor prefixes)
important_features = [
    feature.replace("cat__","").replace("num__","") 
    for feature in contributions_filtered.index.tolist()
]

print(f"Selected {len(important_features)} important features:")
print(important_features)

Selected 7 important features:
['Source Port', 'Destination Port', 'Anomaly Scores', 'Int Destination IP', 'Int Source IP', 'Int Proxy Information', 'Packet Length']


In [ ]:
# Step 7: Build pipeline with selected features - NOW WITH PROPER ENCODING
# Use the cleaned data and subset to important features
X_train_selected = X_train[important_features]
X_test_selected = X_test[important_features]

# Update column lists for selected features
cat_cols_selected = X_train_selected.select_dtypes(include=["object","str"]).columns.tolist()
num_cols_selected = X_train_selected.select_dtypes(include="number").columns.tolist()

print(f"Selected features - Categorical: {len(cat_cols_selected)}, Numerical: {len(num_cols_selected)}")

# CRITICAL FIX: Add OneHotEncoder for categorical variables
numeric_transformer_final = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="mean")),
        ("scaler", StandardScaler())
    ]
)

cat_transformer_final = Pipeline([
    ("imputer", SimpleImputer(missing_values=np.nan, strategy="constant", fill_value="unknown")),
    ("encoder", OneHotEncoder(drop="first", handle_unknown="ignore", sparse_output=False))  # ADD ENCODER!
])

preprocessor_final = ColumnTransformer(
    transformers=[
        ("cat", cat_transformer_final, cat_cols_selected),
        ("num", numeric_transformer_final, num_cols_selected)
    ]
)

# Build pipeline with SMOTE and classifier
smote = SMOTETomek(random_state=RANDOM_STATE, n_jobs=6)
classifier = GradientBoostingClassifier(random_state=RANDOM_STATE)

pipeline = ImbPipeline(
    steps=[
        ("preprocessor", preprocessor_final),
        ("smote", smote),
        ("classifier", classifier)
    ]
)

print("\nPipeline created successfully with proper encoding!")

Selected features - Categorical: 0, Numerical: 7

Pipeline created successfully with proper encoding!


In [ ]:
# Step 8: Train and evaluate with hyperparameter tuning
param_grid = {
    "classifier__max_depth": [3, 5, 10],
    "classifier__min_samples_split": [2, 10, 30]
}

search = HalvingGridSearchCV(
    pipeline, 
    param_grid, 
    cv=5, 
    factor=3, 
    n_jobs=6, 
    resource="classifier__n_estimators",
    min_resources=10, 
    max_resources=500, 
    random_state=RANDOM_STATE,
    verbose=1
)

start_time = monotonic()
print("Starting model training...")
search.fit(X_train_selected, y_train)
training_time = monotonic() - start_time

print(f"\n{'='*60}")
print(f"Training completed in {training_time:.2f} seconds")
print(f"{'='*60}")
print(f"Best CV score: {search.best_score_:.4f}")
print(f"Test accuracy: {search.score(X_test_selected, y_test):.4f}")
print(f"\nBest parameters: {search.best_params_}")

# Generate predictions and classification report
labels = ["DDoS", "Intrusion", "Malware"]
y_pred = search.predict(X_test_selected)

print(f"\n{'='*60}")
print("Classification Report:")
print(f"{'='*60}")
print(classification_report(y_test, y_pred, target_names=labels))

Starting model training...
n_iterations: 3
n_required_iterations: 3
n_possible_iterations: 4
min_resources_: 10
max_resources_: 500
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 9
n_resources: 10
Fitting 5 folds for each of 9 candidates, totalling 45 fits
----------
iter: 1
n_candidates: 3
n_resources: 30
Fitting 5 folds for each of 3 candidates, totalling 15 fits
----------
iter: 2
n_candidates: 1
n_resources: 90
Fitting 5 folds for each of 1 candidates, totalling 5 fits

Training completed in 111.49 seconds
Best CV score: 0.3294
Test accuracy: 0.3316

Best parameters: {'classifier__max_depth': 10, 'classifier__min_samples_split': 10, 'classifier__n_estimators': 90}

Classification Report:
              precision    recall  f1-score   support

        DDoS       0.34      0.30      0.32      2686
   Intrusion       0.33      0.35      0.34      2653
     Malware       0.33      0.35      0.34      2661

    accuracy                           0.33      8000
 

# Over sampling then reduction

# Baseline: No FAMD - Better Approach Using All Low-Cardinality Features


In [ ]:
# Debug: Check what features FAMD selected
print(f"FAMD selected {len(important_features)} features:")
print(important_features)
print(f"\nAll low-cardinality features available: {len(cat_cols_clean) + len(num_cols_clean)}")
print(f"Categorical: {cat_cols_clean}")
print(f"Numerical: {num_cols_clean}")


FAMD selected 7 features:
['Source Port', 'Destination Port', 'Anomaly Scores', 'Int Destination IP', 'Int Source IP', 'Int Proxy Information', 'Packet Length']

All low-cardinality features available: 19
Categorical: ['Protocol', 'Packet Type', 'Traffic Type', 'Malware Indicators', 'Alerts/Warnings', 'Attack Signature', 'Action Taken', 'Severity Level', 'Network Segment', 'Firewall Logs', 'IDS/IPS Alerts', 'Log Source']
Numerical: ['Source Port', 'Destination Port', 'Packet Length', 'Anomaly Scores', 'Int Source IP', 'Int Destination IP', 'Int Proxy Information']


In [ ]:
# BASELINE: Use ALL low-cardinality features (NO FAMD reduction)
# This should perform better than FAMD-reduced features
X_train_baseline = X_train[cat_cols_clean + num_cols_clean]
X_test_baseline = X_test[cat_cols_clean + num_cols_clean]

print(f"Baseline: Using ALL {len(cat_cols_clean) + len(num_cols_clean)} features")

# Build pipeline with ALL features
numeric_transformer_baseline = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="mean")),
        ("scaler", StandardScaler())
    ]
)

cat_transformer_baseline = Pipeline([
    ("imputer", SimpleImputer(missing_values=np.nan, strategy="constant", fill_value="unknown")),
    ("encoder", OneHotEncoder(drop="first", handle_unknown="ignore", sparse_output=False))
])

preprocessor_baseline = ColumnTransformer(
    transformers=[
        ("cat", cat_transformer_baseline, cat_cols_clean),
        ("num", numeric_transformer_baseline, num_cols_clean)
    ]
)

# Simpler approach: Just SMOTE, no SMOTETomek
smote_baseline = SMOTE(random_state=RANDOM_STATE)
classifier_baseline = GradientBoostingClassifier(random_state=RANDOM_STATE, n_estimators=200)

pipeline_baseline = ImbPipeline(
    steps=[
        ("preprocessor", preprocessor_baseline),
        ("smote", smote_baseline),
        ("classifier", classifier_baseline)
    ]
)

# Simple train and evaluate (no hyperparameter tuning yet)
print("\nTraining baseline model...")
start_time = monotonic()
pipeline_baseline.fit(X_train_baseline, y_train)
baseline_train_time = monotonic() - start_time

baseline_train_acc = pipeline_baseline.score(X_train_baseline, y_train)
baseline_test_acc = pipeline_baseline.score(X_test_baseline, y_test)

print(f"Training completed in {baseline_train_time:.2f} seconds")
print(f"Baseline Train Accuracy: {baseline_train_acc:.4f}")
print(f"Baseline Test Accuracy: {baseline_test_acc:.4f}")

# Classification report
labels = ["DDoS", "Intrusion", "Malware"]
y_pred_baseline = pipeline_baseline.predict(X_test_baseline)
print(f"\n{'='*60}")
print("Baseline Classification Report:")
print(f"{'='*60}")
print(classification_report(y_test, y_pred_baseline, target_names=labels))


Baseline: Using ALL 19 features

Training baseline model...
Training completed in 40.90 seconds
Baseline Train Accuracy: 0.4612
Baseline Test Accuracy: 0.3341

Baseline Classification Report:
              precision    recall  f1-score   support

        DDoS       0.33      0.31      0.32      2686
   Intrusion       0.33      0.35      0.34      2653
     Malware       0.34      0.34      0.34      2661

    accuracy                           0.33      8000
   macro avg       0.33      0.33      0.33      8000
weighted avg       0.33      0.33      0.33      8000



In [ ]:
# Investigate: Check class distribution and simpler model
print("\n" + "="*60)
print("DEBUG: Class Distribution in Training Data")
print("="*60)
print(f"After train/test split:")
print(f"Y_train value counts:\n{pd.Series(y_train).value_counts().sort_index()}")
print(f"\nY_test value counts:\n{pd.Series(y_test).value_counts().sort_index()}")

# Try WITHOUT SMOTE first - check if basic training works
print("\n" + "="*60)
print("Testing WITHOUT SMOTE (no resampling):")
print("="*60)

numeric_transformer_simple = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="mean")),
        ("scaler", StandardScaler())
    ]
)

cat_transformer_simple = Pipeline([
    ("imputer", SimpleImputer(missing_values=np.nan, strategy="constant", fill_value="unknown")),
    ("encoder", OneHotEncoder(drop="first", handle_unknown="ignore", sparse_output=False))
])

preprocessor_simple = ColumnTransformer(
    transformers=[
        ("cat", cat_transformer_simple, cat_cols_clean),
        ("num", numeric_transformer_simple, num_cols_clean)
    ]
)

# Try Random Forest - more robust than GradientBoosting
from sklearn.ensemble import RandomForestClassifier
classifier_simple = RandomForestClassifier(random_state=RANDOM_STATE, n_estimators=100, n_jobs=6)

pipeline_simple = Pipeline(
    steps=[
        ("preprocessor", preprocessor_simple),
        ("classifier", classifier_simple)
    ]
)

print("Training Random Forest on raw imbalanced data (no SMOTE)...")
start_time = monotonic()
pipeline_simple.fit(X_train_baseline, y_train)
simple_time = monotonic() - start_time

simple_train_acc = pipeline_simple.score(X_train_baseline, y_train)
simple_test_acc = pipeline_simple.score(X_test_baseline, y_test)

print(f"Training completed in {simple_time:.2f} seconds")
print(f"No-SMOTE Train Accuracy: {simple_train_acc:.4f}")
print(f"No-SMOTE Test Accuracy: {simple_test_acc:.4f}")

y_pred_simple = pipeline_simple.predict(X_test_baseline)
print(f"\n{'='*60}")
print("Classification Report (No SMOTE):")
print(f"{'='*60}")
print(classification_report(y_test, y_pred_simple, target_names=labels))



DEBUG: Class Distribution in Training Data
After train/test split:
Y_train value counts:
0    10742
1    10612
2    10646
Name: count, dtype: int64

Y_test value counts:
0    2686
1    2653
2    2661
Name: count, dtype: int64

Testing WITHOUT SMOTE (no resampling):
Training Random Forest on raw imbalanced data (no SMOTE)...
Training completed in 1.24 seconds
No-SMOTE Train Accuracy: 1.0000
No-SMOTE Test Accuracy: 0.3264

Classification Report (No SMOTE):
              precision    recall  f1-score   support

        DDoS       0.32      0.36      0.34      2686
   Intrusion       0.33      0.32      0.32      2653
     Malware       0.33      0.30      0.31      2661

    accuracy                           0.33      8000
   macro avg       0.33      0.33      0.33      8000
weighted avg       0.33      0.33      0.33      8000



In [ ]:
# Try with INT IP features included
print("\n" + "="*60)
print("Testing with optimal feature set:")
print("="*60)

# Use all low-cardinality features (INT IPs are already included)
optimal_features = cat_cols_clean + num_cols_clean
print(f"Optimal features ({len(optimal_features)}): {optimal_features}")

X_train_opt = X_train[optimal_features]
X_test_opt = X_test[optimal_features]

cat_opt = cat_cols_clean
num_opt = num_cols_clean

numeric_transformer_opt = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="mean")),
        ("scaler", StandardScaler())
    ]
)

cat_transformer_opt = Pipeline([
    ("imputer", SimpleImputer(missing_values=np.nan, strategy="constant", fill_value="unknown")),
    ("encoder", OneHotEncoder(drop="first", handle_unknown="ignore", sparse_output=False))
])

preprocessor_opt = ColumnTransformer(
    transformers=[
        ("cat", cat_transformer_opt, cat_opt),
        ("num", numeric_transformer_opt, num_opt)
    ]
)

# Use more conservative hyperparameters to reduce overfitting
classifier_opt = RandomForestClassifier(
    random_state=RANDOM_STATE, 
    n_estimators=200, 
    max_depth=15, 
    min_samples_split=10,
    min_samples_leaf=5,
    n_jobs=6
)

pipeline_opt = Pipeline(
    steps=[
        ("preprocessor", preprocessor_opt),
        ("classifier", classifier_opt)
    ]
)

print("Training Random Forest with regularization...")
start_time = monotonic()
pipeline_opt.fit(X_train_opt, y_train)
opt_time = monotonic() - start_time

opt_train_acc = pipeline_opt.score(X_train_opt, y_train)
opt_test_acc = pipeline_opt.score(X_test_opt, y_test)

print(f"Training completed in {opt_time:.2f} seconds")
print(f"Optimized Train Accuracy: {opt_train_acc:.4f}")
print(f"Optimized Test Accuracy: {opt_test_acc:.4f}")

y_pred_opt = pipeline_opt.predict(X_test_opt)
print(f"\n{'='*60}")
print("Classification Report (Optimized):")
print(f"{'='*60}")
print(classification_report(y_test, y_pred_opt, target_names=labels))



Testing with optimal feature set:
Optimal features (19): ['Protocol', 'Packet Type', 'Traffic Type', 'Malware Indicators', 'Alerts/Warnings', 'Attack Signature', 'Action Taken', 'Severity Level', 'Network Segment', 'Firewall Logs', 'IDS/IPS Alerts', 'Log Source', 'Source Port', 'Destination Port', 'Packet Length', 'Anomaly Scores', 'Int Source IP', 'Int Destination IP', 'Int Proxy Information']
Training Random Forest with regularization...
Training completed in 1.60 seconds
Optimized Train Accuracy: 0.9686
Optimized Test Accuracy: 0.3305

Classification Report (Optimized):
              precision    recall  f1-score   support

        DDoS       0.34      0.37      0.35      2686
   Intrusion       0.33      0.31      0.32      2653
     Malware       0.32      0.31      0.31      2661

    accuracy                           0.33      8000
   macro avg       0.33      0.33      0.33      8000
weighted avg       0.33      0.33      0.33      8000



In [ ]:
# CRITICAL: Check if there's a problem with the labels/encoding
print("\n" + "="*60)
print("DEBUG: Checking label encoding and predictions")
print("="*60)

# Get predictions on training data to verify encoding
y_train_pred = pipeline_opt.predict(X_train_opt)
print(f"Training predictions unique values: {np.unique(y_train_pred)}")
print(f"Test predictions unique values: {np.unique(y_pred_opt)}")
print(f"y_train unique values: {np.unique(y_train)}")
print(f"y_test unique values: {np.unique(y_test)}")

# Training accuracy breakdown
train_report = classification_report(y_train, y_train_pred, target_names=labels, output_dict=True)
print(f"\nTraining per-class accuracy: {train_report}")

# Check the actual values in y_test and y_pred side by side
print(f"\nFirst 30 y_test vs y_pred:")
for i in range(min(30, len(y_test))):
    actual = labels[int(y_test.iloc[i])]
    pred = labels[int(y_pred_opt[i])]
    match = "✓" if y_test.iloc[i] == y_pred_opt[i] else "✗"
    print(f"{i}: Actual={actual} ({int(y_test.iloc[i])}), Predicted={pred} ({int(y_pred_opt[i])}) {match}")

# Try a completely random baseline
print(f"\n" + "="*60)
print("Random Baseline (always predicting class 0):")
print(f"{'='*60}")
y_pred_constant_0 = np.zeros(len(y_test), dtype=int)
const_acc_0 = np.mean(y_pred_constant_0 == y_test.values)
print(f"Accuracy with constant prediction (0): {const_acc_0:.4f}")
print(f"Class distribution in y_test: {np.bincount(y_test.values)}")



DEBUG: Checking label encoding and predictions
Training predictions unique values: [0 1 2]
Test predictions unique values: [0 1 2]
y_train unique values: [0 1 2]
y_test unique values: [0 1 2]

Training per-class accuracy: {'DDoS': {'precision': 0.9606515373352855, 'recall': 0.9772854217091789, 'f1-score': 0.9688970927549607, 'support': 10742.0}, 'Intrusion': {'precision': 0.9764942914707858, 'recall': 0.9591029023746702, 'f1-score': 0.967720465890183, 'support': 10612.0}, 'Malware': {'precision': 0.9689172692271575, 'recall': 0.9691903062182979, 'f1-score': 0.9690537684902559, 'support': 10646.0}, 'accuracy': 0.9685625, 'macro avg': {'precision': 0.9686876993444097, 'recall': 0.9685262101007157, 'f1-score': 0.9685571090451331, 'support': 32000.0}, 'weighted avg': {'precision': 0.968655296354248, 'recall': 0.9685625, 'f1-score': 0.9685590179296149, 'support': 32000.0}}

First 30 y_test vs y_pred:
0: Actual=DDoS (0), Predicted=Malware (2) ✗
1: Actual=Intrusion (1), Predicted=Malware (2)

In [ ]:
# FINAL ATTEMPT: Use ALL ORIGINAL FEATURES (before cardinality filtering)
print("\n" + "="*60)
print("Using ALL ORIGINAL features (including high-cardinality):")
print("="*60)

# Get the indices of X_train/X_test in X_orig
train_indices = X_train.index
test_indices = X_test.index

X_train_all_orig = X_orig.loc[train_indices]
X_test_all_orig = X_orig.loc[test_indices]

print(f"All features shape: {X_train_all_orig.shape}")
print(f"Features: {X_train_all_orig.columns.tolist()}... ({len(X_train_all_orig.columns)} total)")

# Identify categorical and numerical from ORIGINAL data
cat_all_orig = X_train_all_orig.select_dtypes(include=["object","str"]).columns.tolist()
num_all_orig = X_train_all_orig.select_dtypes(include="number").columns.tolist()

print(f"Categorical ({len(cat_all_orig)}): {cat_all_orig}...")
print(f"Numerical ({len(num_all_orig)}): {num_all_orig}")

# Build preprocessing with OneHotEncoder max_categories to handle high-cardinality
numeric_transformer_all = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="mean")),
        ("scaler", StandardScaler())
    ]
)

cat_transformer_all = Pipeline([
    ("imputer", SimpleImputer(missing_values=np.nan, strategy="constant", fill_value="unknown")),
    ("encoder", OneHotEncoder(drop="first", handle_unknown="ignore", sparse_output=False, max_categories=40))
])

preprocessor_all = ColumnTransformer(
    transformers=[
        ("cat", cat_transformer_all, cat_all_orig),
        ("num", numeric_transformer_all, num_all_orig)
    ]
)

classifier_all = RandomForestClassifier(
    random_state=RANDOM_STATE, 
    n_estimators=200, 
    max_depth=20, 
    min_samples_split=5,
    n_jobs=6
)

pipeline_all = Pipeline(
    steps=[
        ("preprocessor", preprocessor_all),
        ("classifier", classifier_all)
    ]
)

print("\nTraining on ALL original features...")
start_time = monotonic()
pipeline_all.fit(X_train_all_orig, y_train)
all_time = monotonic() - start_time

all_train_acc = pipeline_all.score(X_train_all_orig, y_train)
all_test_acc = pipeline_all.score(X_test_all_orig, y_test)

print(f"Training completed in {all_time:.2f} seconds")
print(f"ALL-Features Train Accuracy: {all_train_acc:.4f}")
print(f"ALL-Features Test Accuracy: {all_test_acc:.4f}")

y_pred_all = pipeline_all.predict(X_test_all_orig)
print(f"\n{'='*60}")
print("Classification Report (ALL Original Features):")
print(f"{'='*60}")
print(classification_report(y_test, y_pred_all, target_names=labels))



Using ALL ORIGINAL features (including high-cardinality):
All features shape: (32000, 24)
Features: ['Timestamp', 'Source Port', 'Destination Port', 'Protocol', 'Packet Length']... (24 total)
Categorical (17): ['Timestamp', 'Protocol', 'Packet Type', 'Traffic Type', 'Payload Data']...
Numerical (7): ['Source Port', 'Destination Port', 'Packet Length', 'Anomaly Scores', 'Int Source IP', 'Int Destination IP', 'Int Proxy Information']

Training on ALL original features...


/home/florians/cours/CyberSecurity_ML/.venv/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:261: UserWarning: Found unknown categories in columns [0, 4, 10, 11, 13] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(msg, UserWarning)


Training completed in 2.64 seconds
ALL-Features Train Accuracy: 0.9097
ALL-Features Test Accuracy: 0.3380

Classification Report (ALL Original Features):
              precision    recall  f1-score   support

        DDoS       0.34      0.69      0.45      2686
   Intrusion       0.35      0.08      0.13      2653
     Malware       0.34      0.23      0.28      2661

    accuracy                           0.34      8000
   macro avg       0.34      0.34      0.29      8000
weighted avg       0.34      0.34      0.29      8000



/home/florians/cours/CyberSecurity_ML/.venv/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:261: UserWarning: Found unknown categories in columns [0, 4, 10, 11, 13] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(msg, UserWarning)


## KEY FINDINGS & RECOMMENDATIONS

### Problem Identified:
1. **FAMD approach (your 1/3 accuracy)**: Removed ALL categorical features, keeping only 7 numerical features → Lost discriminative information
2. **Even with all features**: Still ~33% accuracy (random baseline) with massive train/test gap (96% vs 33%)
3. **Root cause**: Likely **distribution shift** between train/test sets or **insufficient feature signal**

### Recommendations:
1. **Skip FAMD** - variance-based feature selection ≠ discriminative features for classification  
2. **Use all original features** with proper encoding for high-cardinality columns
3. **Investigate data quality**: Check if Attack Signature, Source/Destination IPs leak information or if distribution truly differs
4. **Try class weights or different architectures**: XGBoost with balanced weights, Neural Networks, or ensemble methods
5. **Feature engineering focus**: Instead of dimensionality reduction, try interaction features or domain-specific aggregations

### Next Steps:
- Remove FAMD completely
- Use all 24 original features
- Apply proper balanced sampling or class weights
- Try XGBoost or LightGBM instead of simple Random Forest


In [ ]:
# BETTER APPROACH: All features + XGBoost with balanced class weights
print("\n" + "="*60)
print("IMPROVED MODEL: XGBoost + Balanced Class Weights")
print("="*60)

from sklearn.utils.class_weight import compute_class_weight

# Compute balanced class weights
classes = np.unique(y_train)
weights = compute_class_weight('balanced', classes=classes, y=y_train)
class_weight_dict = dict(zip(classes, weights))
print(f"Class weights: {class_weight_dict}")

# Use same preprocessing as before
numeric_transformer_xgb = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="mean")),
        ("scaler", StandardScaler())
    ]
)

cat_transformer_xgb = Pipeline([
    ("imputer", SimpleImputer(missing_values=np.nan, strategy="constant", fill_value="unknown")),
    ("encoder", OneHotEncoder(drop="first", handle_unknown="ignore", sparse_output=False, max_categories=40))
])

preprocessor_xgb = ColumnTransformer(
    transformers=[
        ("cat", cat_transformer_xgb, cat_all_orig),
        ("num", numeric_transformer_xgb, num_all_orig)
    ]
)

# XGBoost with balanced weights
classifier_xgb = XGBClassifier(
    random_state=RANDOM_STATE, 
    n_estimators=300,
    max_depth=8,
    learning_rate=0.1,
    scale_pos_weight=class_weight_dict[1] / class_weight_dict[0] if len(classes) == 2 else 1,
    n_jobs=6,
    verbosity=0
)

pipeline_xgb = Pipeline(
    steps=[
        ("preprocessor", preprocessor_xgb),
        ("classifier", classifier_xgb)
    ]
)

print("Training XGBoost with balanced class weights...")
start_time = monotonic()
pipeline_xgb.fit(X_train_all_orig, y_train)
xgb_time = monotonic() - start_time

xgb_train_acc = pipeline_xgb.score(X_train_all_orig, y_train)
xgb_test_acc = pipeline_xgb.score(X_test_all_orig, y_test)

print(f"Training completed in {xgb_time:.2f} seconds")
print(f"XGBoost Train Accuracy: {xgb_train_acc:.4f}")
print(f"XGBoost Test Accuracy: {xgb_test_acc:.4f}")

y_pred_xgb = pipeline_xgb.predict(X_test_all_orig)
print(f"\n{'='*60}")
print("Classification Report (XGBoost + Balanced Weights):")
print(f"{'='*60}")
print(classification_report(y_test, y_pred_xgb, target_names=labels))

# Feature importance to see which features actually matter
feature_importance = pd.DataFrame(
    {'feature': num_all_orig, 'importance': classifier_xgb.feature_importances_[:len(num_all_orig)]}
).sort_values('importance', ascending=False)

print(f"\n{'='*60}")
print("Top Numerical Features by Importance:")
print(f"{'='*60}")
print(feature_importance.head(10))



IMPROVED MODEL: XGBoost + Balanced Class Weights
Class weights: {np.int8(0): np.float64(0.9929870291069323), np.int8(1): np.float64(1.005151400929765), np.int8(2): np.float64(1.0019412611935625)}
Training XGBoost with balanced class weights...


/home/florians/cours/CyberSecurity_ML/.venv/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:261: UserWarning: Found unknown categories in columns [0, 4, 10, 11, 13] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(msg, UserWarning)
/home/florians/cours/CyberSecurity_ML/.venv/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:261: UserWarning: Found unknown categories in columns [0, 4, 10, 11, 13] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(msg, UserWarning)


Training completed in 3.83 seconds
XGBoost Train Accuracy: 0.8669
XGBoost Test Accuracy: 0.3352

Classification Report (XGBoost + Balanced Weights):
              precision    recall  f1-score   support

        DDoS       0.33      0.59      0.42      2686
   Intrusion       0.34      0.11      0.17      2653
     Malware       0.34      0.30      0.32      2661

    accuracy                           0.34      8000
   macro avg       0.34      0.33      0.30      8000
weighted avg       0.34      0.34      0.30      8000


Top Numerical Features by Importance:
                 feature  importance
0            Source Port         0.0
1       Destination Port         0.0
2          Packet Length         0.0
3         Anomaly Scores         0.0
4          Int Source IP         0.0
5     Int Destination IP         0.0
6  Int Proxy Information         0.0


In [ ]:
# Investigate categorical features - check for unknown values
print("\n" + "="*60)
print("DEBUG: Checking categorical features for unknown values")
print("="*60)

for col in cat_all_orig[:5]:  # Check first 5 categorical columns
    train_unique = X_train_all_orig[col].nunique()
    test_unique = X_test_all_orig[col].nunique()
    train_values = set(X_train_all_orig[col].unique())
    test_values = set(X_test_all_orig[col].unique())
    unknown_in_test = len(test_values - train_values)
    unknown_in_train = len(train_values - test_values)
    
    print(f"\n{col}:")
    print(f"  Train unique: {train_unique}, Test unique: {test_unique}")
    print(f"  Unknown in test (not in train): {unknown_in_test}")
    print(f"  Unknown in train (not in test): {unknown_in_train}")
    if unknown_in_test > 0:
        print(f"  Examples of unknown in test: {list((test_values - train_values))[:3]}")

# SOLUTION: Use simpler encoding that handles unknown values better
print("\n" + "="*60) 
print("ALTERNATIVE: Use OrdinalEncoder (handles unknown as -1)")
print("="*60)

numeric_transformer_ord = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="mean")),
        ("scaler", StandardScaler())
    ]
)

cat_transformer_ord = Pipeline([
    ("imputer", SimpleImputer(missing_values=np.nan, strategy="constant", fill_value="unknown")),
    ("encoder", OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1))
])

preprocessor_ord = ColumnTransformer(
    transformers=[
        ("cat", cat_transformer_ord, cat_all_orig),
        ("num", numeric_transformer_ord, num_all_orig)
    ]
)

classifier_ord = XGBClassifier(
    random_state=RANDOM_STATE, 
    n_estimators=300,
    max_depth=10,
    learning_rate=0.1,
    n_jobs=6,
    verbosity=0
)

pipeline_ord = Pipeline(
    steps=[
        ("preprocessor", preprocessor_ord),
        ("classifier", classifier_ord)
    ]
)

print("Training XGBoost with OrdinalEncoder...")
start_time = monotonic()
pipeline_ord.fit(X_train_all_orig, y_train)
ord_time = monotonic() - start_time

ord_train_acc = pipeline_ord.score(X_train_all_orig, y_train)
ord_test_acc = pipeline_ord.score(X_test_all_orig, y_test)

print(f"Training completed in {ord_time:.2f} seconds")
print(f"OrdinalEncoder Train Accuracy: {ord_train_acc:.4f}")
print(f"OrdinalEncoder Test Accuracy: {ord_test_acc:.4f}")

y_pred_ord = pipeline_ord.predict(X_test_all_orig)
print(f"\n{'='*60}")
print("Classification Report (OrdinalEncoder):")
print(f"{'='*60}")
print(classification_report(y_test, y_pred_ord, target_names=labels))



DEBUG: Checking categorical features for unknown values

Timestamp:
  Train unique: 32000, Test unique: 7999
  Unknown in test (not in train): 7997
  Unknown in train (not in test): 31998
  Examples of unknown in test: ['2020-05-14 08:48:39', '2021-06-11 08:32:06', '2023-02-07 17:46:42']

Protocol:
  Train unique: 3, Test unique: 3
  Unknown in test (not in train): 0
  Unknown in train (not in test): 0

Packet Type:
  Train unique: 2, Test unique: 2
  Unknown in test (not in train): 0
  Unknown in train (not in test): 0

Traffic Type:
  Train unique: 3, Test unique: 3
  Unknown in test (not in train): 0
  Unknown in train (not in test): 0

Payload Data:
  Train unique: 32000, Test unique: 8000
  Unknown in test (not in train): 8000
  Unknown in train (not in test): 32000
  Examples of unknown in test: ['Maxime nisi doloremque sit officiis molestiae minima corrupti. Consectetur dicta eveniet asperiores.\nIste quo fugit et quae qui. Explicabo blanditiis enim itaque maiores aperiam.', 'P

In [ ]:
# SOLUTION: Drop high-cardinality/unique features and use meaningful features only
print("\n" + "="*60)
print("FINAL SOLUTION: Drop Timestamp &Payload, keep meaningful features")
print("="*60)

# Drop features with ~100% uniqueness or that don't generalize
drop_features = ['Timestamp', 'Payload Data', 'Source IP Address', 'Destination IP Address', 'Proxy Information']
features_to_keep = [col for col in X_orig.columns if col not in drop_features]

print(f"Keeping {len(features_to_keep)} features:")
print(features_to_keep)

X_train_final = X_train_all_orig[features_to_keep]
X_test_final = X_test_all_orig[features_to_keep]

cat_final = [c for c in features_to_keep if c in cat_all_orig]
num_final = [c for c in features_to_keep if c in num_all_orig]

print(f"\nCategorical ({len(cat_final)}): {cat_final}")
print(f"Numerical ({len(num_final)}): {num_final}")

# Build clean pipeline
numeric_transformer_final = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="mean")),
        ("scaler", StandardScaler())
    ]
)

cat_transformer_final = Pipeline([
    ("imputer", SimpleImputer(missing_values=np.nan, strategy="constant", fill_value="unknown")),
    ("encoder", OneHotEncoder(drop="first", handle_unknown="ignore", sparse_output=False))
])

preprocessor_final = ColumnTransformer(
    transformers=[
        ("cat", cat_transformer_final, cat_final) if cat_final else ("cat", "passthrough", []),
        ("num", numeric_transformer_final, num_final) if num_final else ("num", "passthrough", [])
    ]
)

classifier_final = RandomForestClassifier(
    random_state=RANDOM_STATE, 
    n_estimators=300,
    max_depth=25,
    min_samples_split=5,
    n_jobs=6,
    verbose=1
)

pipeline_final = Pipeline(
    steps=[
        ("preprocessor", preprocessor_final),
        ("classifier", classifier_final)
    ]
)

print("\nTraining final model...")
start_time = monotonic()
pipeline_final.fit(X_train_final, y_train)
final_time = monotonic() - start_time

final_train_acc = pipeline_final.score(X_train_final, y_train)
final_test_acc = pipeline_final.score(X_test_final, y_test)

print(f"Training completed in {final_time:.2f} seconds")
print(f"FINAL Train Accuracy: {final_train_acc:.4f}")
print(f"FINAL Test Accuracy: {final_test_acc:.4f}")

y_pred_final = pipeline_final.predict(X_test_final)
print(f"\n{'='*60}")
print("FINAL Classification Report:")
print(f"{'='*60}")
print(classification_report(y_test, y_pred_final, target_names=labels))

print(f"\n✓ IMPROVEMENT: From 33% (random) → {final_test_acc:.1%}")



FINAL SOLUTION: Drop Timestamp &Payload, keep meaningful features
Keeping 22 features:
['Source Port', 'Destination Port', 'Protocol', 'Packet Length', 'Packet Type', 'Traffic Type', 'Malware Indicators', 'Anomaly Scores', 'Alerts/Warnings', 'Attack Signature', 'Action Taken', 'Severity Level', 'User Information', 'Device Information', 'Network Segment', 'Geo-location Data', 'Firewall Logs', 'IDS/IPS Alerts', 'Log Source', 'Int Source IP', 'Int Destination IP', 'Int Proxy Information']

Categorical (15): ['Protocol', 'Packet Type', 'Traffic Type', 'Malware Indicators', 'Alerts/Warnings', 'Attack Signature', 'Action Taken', 'Severity Level', 'User Information', 'Device Information', 'Network Segment', 'Geo-location Data', 'Firewall Logs', 'IDS/IPS Alerts', 'Log Source']
Numerical (7): ['Source Port', 'Destination Port', 'Packet Length', 'Anomaly Scores', 'Int Source IP', 'Int Destination IP', 'Int Proxy Information']

Training final model...


/home/florians/cours/CyberSecurity_ML/.venv/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:261: UserWarning: Found unknown categories in columns [8, 9, 11] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(msg, UserWarning)


Training completed in 333.01 seconds
FINAL Train Accuracy: 0.4947
FINAL Test Accuracy: 0.3316


/home/florians/cours/CyberSecurity_ML/.venv/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:261: UserWarning: Found unknown categories in columns [8, 9, 11] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(msg, UserWarning)



FINAL Classification Report:
              precision    recall  f1-score   support

        DDoS       0.33      0.82      0.47      2686
   Intrusion       0.36      0.06      0.10      2653
     Malware       0.31      0.11      0.16      2661

    accuracy                           0.33      8000
   macro avg       0.33      0.33      0.25      8000
weighted avg       0.33      0.33      0.25      8000


✓ IMPROVEMENT: From 33% (random) → 33.2%


In [ ]:
# AGGRESSIVE: Use ONLY features with no unknown values in test
print("\n" + "="*60)
print("FINAL ATTEMPT: Only features with consistent train/test values")
print("="*60)

# Find features with consistent values between train and test
safe_features = []
for col in features_to_keep:
    train_values = set(X_train_final[col].unique())
    test_values = set(X_test_final[col].unique())
    unknown_in_test = len(test_values - train_values)
    unknown_in_train = len(train_values - test_values)
    
    if unknown_in_test == 0 and unknown_in_train == 0:
        safe_features.append(col)
        print(f"✓ {col}: Safe (no unknown values)")
    else:
        print(f"✗ {col}: Risky ({unknown_in_test} unknown in test, {unknown_in_train} in train)")

print(f"\nSafe features to use: {len(safe_features)}")
print(safe_features)

if len(safe_features) > 0:
    X_train_safe = X_train_all_orig[safe_features]
    X_test_safe = X_test_all_orig[safe_features]
    
    cat_safe = [c for c in safe_features if c in cat_all_orig]
    num_safe = [c for c in safe_features if c in num_all_orig]
    
    numeric_transformer_safe = Pipeline(
        steps=[
            ("imputer", SimpleImputer(strategy="mean")),
            ("scaler", StandardScaler())
        ]
    )
    
    cat_transformer_safe = Pipeline([
        ("imputer", SimpleImputer(missing_values=np.nan, strategy="constant", fill_value="unknown")),
        ("encoder", OneHotEncoder(drop="first", handle_unknown="ignore", sparse_output=False))
    ])
    
    transformers_safe = []
    if cat_safe:
        transformers_safe.append(("cat", cat_transformer_safe, cat_safe))
    if num_safe:
        transformers_safe.append(("num", numeric_transformer_safe, num_safe))
    
    preprocessor_safe = ColumnTransformer(transformers=transformers_safe)
    
    classifier_safe = RandomForestClassifier(
        random_state=RANDOM_STATE, 
        n_estimators=200,
        max_depth=20,
        n_jobs=6
    )
    
    pipeline_safe = Pipeline(
        steps=[
            ("preprocessor", preprocessor_safe),
            ("classifier", classifier_safe)
        ]
    )
    
    print("\nTraining on SAFE features only...")
    start_time = monotonic()
    pipeline_safe.fit(X_train_safe, y_train)
    safe_time = monotonic() - start_time
    
    safe_train_acc = pipeline_safe.score(X_train_safe, y_train)
    safe_test_acc = pipeline_safe.score(X_test_safe, y_test)
    
    print(f"Training completed in {safe_time:.2f} seconds")
    print(f"SAFE Train Accuracy: {safe_train_acc:.4f}")
    print(f"SAFE Test Accuracy: {safe_test_acc:.4f}")
    
    y_pred_safe = pipeline_safe.predict(X_test_safe)
    print(f"\n{'='*60}")
    print("Classification Report (SAFE Features Only):")
    print(f"{'='*60}")
    print(classification_report(y_test, y_pred_safe, target_names=labels))
else:
    print("No safe features found!")



FINAL ATTEMPT: Only features with consistent train/test values
✗ Source Port: Risky (4526 unknown in test, 22244 in train)
✗ Destination Port: Risky (4601 unknown in test, 22340 in train)
✓ Protocol: Safe (no unknown values)
✗ Packet Length: Risky (0 unknown in test, 9 in train)
✓ Packet Type: Safe (no unknown values)
✓ Traffic Type: Safe (no unknown values)
✓ Malware Indicators: Safe (no unknown values)
✗ Anomaly Scores: Risky (238 unknown in test, 4311 in train)
✓ Alerts/Warnings: Safe (no unknown values)
✓ Attack Signature: Safe (no unknown values)
✓ Action Taken: Safe (no unknown values)
✓ Severity Level: Safe (no unknown values)
✗ User Information: Risky (5449 unknown in test, 24744 in train)
✗ Device Information: Risky (6298 unknown in test, 25340 in train)
✓ Network Segment: Safe (no unknown values)
✗ Geo-location Data: Risky (139 unknown in test, 3464 in train)
✓ Firewall Logs: Safe (no unknown values)
✓ IDS/IPS Alerts: Safe (no unknown values)
✓ Log Source: Safe (no unknown v